In [1]:
import os
import sys
import subprocess

from from_root import from_root

sys.path.insert(0, str(from_root("src")))

from utils import get_base_location
from read_and_write_docs import read_jsonl

In [ ]:

corpus      = "Wiki"
data_type   = "test"

# Set NAS so can run on Windows laptop seamlessly
nas_base_loc = get_base_location()

known_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
unknown_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
metadata_loc = f"{nas_base_loc}/datasets/author_verification/{data_type}/metadata.rds"
model_loc = f"{nas_base_loc}/models/Qwen 2.5/Qwen2.5-0.5B-Instruct"
save_loc = f"{nas_base_loc}/paraphrase examples/{corpus}-{data_type}/"

openai_model = "gpt-4.1"
max_tokens   = 5000
temperature  = 0.7
n_samples    = 10

script_loc = str(from_root("scripts", "run_openai_paraphrase_method.py"))

In [6]:
# Load the pre-made problem datasets for speed
problem_dataset_base = f"{nas_base_loc}/datasets/author_verification/{data_type}/{corpus}"
problem_dataset_agg = read_jsonl(f"{problem_dataset_base}/{corpus}_{data_type}_agg.jsonl")

In [7]:
same_probs = problem_dataset_agg[problem_dataset_agg['known_author'] == problem_dataset_agg['unknown_author']].copy()
same_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
same_probs[(same_probs['highest_common_count'] >= 3) & (same_probs['highest_common_count'] <= 10)].head(30)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
601,Vedant vs Vedant,Vedant,Vedant,vedant_text_2,vedant_text_4,10,this Ġis Ġthe Ġproblem Ġwhen Ġindian Ġnational...
236,Mayalld vs Mayalld,Mayalld,Mayalld,mayalld_text_4,mayalld_text_3,10,"Ġunder Ġthe Ġname Ġof Ġthe Ġmaster Ġaccount , ..."
546,Swift&silent vs Swift&silent,Swift&silent,Swift&silent,swift_silent_text_1,swift_silent_text_2,10,"ug ab oy 5 3 5 1 3 6 ,"
506,Snowded vs Snowded,Snowded,Snowded,snowded_text_2,snowded_text_1,8,Ġis Ġa Ġbehavioural Ġissue Ġnot Ġa Ġcontent Ġone
138,KBlott vs KBlott,KBlott,KBlott,kblott_text_3,kblott_text_1,8,Ġevidence Ġto Ġsupport Ġthe Ġpractice Ġof Ġwit...
426,Richard_Daft vs Richard_Daft,Richard_Daft,Richard_Daft,richard_daft_text_2,richard_daft_text_4,7,Ġpre Ġ 1 8 0 0 Ġcricket
588,U21980 vs U21980,U21980,U21980,u21980_text_1,u21980_text_5,7,Ġnx iv m Ġand Ġr ani ere
462,Scheinwerfermann vs Scheinwerfermann,Scheinwerfermann,Scheinwerfermann,scheinwerfermann_text_11,scheinwerfermann_text_10,7,Ġto Ġfind Ġproduction Ġstart Ġand Ġend Ġdates
565,Thekohser vs Thekohser,Thekohser,Thekohser,thekohser_text_10,thekohser_text_11,7,"Ġ 1 6 , 0 0 0"
661,Yoenit vs Yoenit,Yoenit,Yoenit,yoenit_text_4,yoenit_text_2,7,. Ġ' y oen it ' Ġ


In [8]:
diff_probs = problem_dataset_agg[problem_dataset_agg['known_author'] != problem_dataset_agg['unknown_author']].copy()
diff_probs.sort_values(["highest_common_count"], ascending=[False], inplace=True)
diff_probs[(diff_probs['highest_common_count'] >= 3) & (diff_probs['highest_common_count'] <= 10)].head(30)

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
453,Salix_alba vs Sally_Season,Salix_alba,Sally_Season,salix_alba_text_1,sally_season_text_1,6,.Ċ i 'm Ġsorry Ġif Ġi
347,Nocrowx vs Notpietru,Nocrowx,Notpietru,nocrowx_text_5,notpietru_text_2,5,Ġenc yc lo ped ic
227,Mathsci vs Maunus,Mathsci,Maunus,mathsci_text_3,maunus_text_1,5,", Ġa Ġlarge Ġnumber Ġof"
58,IvoShandor vs Jasper_Deng,IvoShandor,Jasper_Deng,ivoshandor_text_4,jasper_deng_text_4,5,Ġas Ġfar Ġas Ġi Ġknow
460,Sally_Season vs Scheinwerfermann,Sally_Season,Scheinwerfermann,sally_season_text_12,scheinwerfermann_text_10,5,Ġyou Ġdon 't Ġwant Ġto
627,VsevolodKrolikov vs WIKI-GUY-16,VsevolodKrolikov,WIKI-GUY-16,vsevolodkrolikov_text_11,wiki_guy_16_text_1,5,.Ċ i Ġdon 't Ġsee
392,PeterSymonds vs Peter_James,PeterSymonds,Peter_James,petersymonds_text_4,peter_james_text_4,5,Ġi Ġdon 't Ġthink Ġit
591,U21980 vs UpDown,U21980,UpDown,u21980_text_1,updown_text_3,5,", Ġbut Ġi Ġdon 't"
413,Pro-Lick vs Protonk,Pro-Lick,Protonk,pro_lick_text_5,protonk_text_13,5,Ġnothing Ġto Ġdo Ġwith Ġthe
305,Nableezy vs Nathan,Nableezy,Nathan,nableezy_text_12,nathan_text_10,5,.Ċ the Ġsimple Ġfact Ġis


In [12]:
same_probs[same_probs['problem'] == 'Mayalld vs Mayalld']

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
236,Mayalld vs Mayalld,Mayalld,Mayalld,mayalld_text_4,mayalld_text_3,10,"Ġunder Ġthe Ġname Ġof Ġthe Ġmaster Ġaccount , ..."
234,Mayalld vs Mayalld,Mayalld,Mayalld,mayalld_text_1,mayalld_text_3,6,.Ċ as Ġfar Ġas Ġi Ġcan
235,Mayalld vs Mayalld,Mayalld,Mayalld,mayalld_text_2,mayalld_text_3,4,.Ċ please Ġnote Ġthat


In [ ]:
tests = {
    "test_01": {"known": "vedant_text_5", "unknown": "vedant_text_4"},
    "test_02": {"known": "vedant_text_2", "unknown": "vedant_text_4"},
    "test_03": {"known": "vedant_text_1", "unknown": "vedant_text_4"},
    "test_04": {"known": "mayalld_text_4", "unknown": "mayalld_text_3"},
    "test_05": {"known": "mayalld_text_1", "unknown": "mayalld_text_3"},
    "test_06": {"known": "mayalld_text_2", "unknown": "mayalld_text_3"},
    "test_07": {"known": "kam_keiser_mail_3", "unknown": "kam_keiser_mail_4"},
    "test_08": {"known": "jeffrey_shankman_mail_2", "unknown": "joannie_williamson_mail_4"},
    "test_09": {"known": "daren_farmer_mail_2", "unknown": "daren_farmer_mail_3"},
    "test_10": {"known": "darron_giron_mail_3", "unknown": "david_delainey_mail_3"}, 
}

In [30]:
# sys.executable.replace("c:\\", "C:/").replace("\\", "/")

In [31]:
# script_loc.replace("C:\\", "C:/").replace("\\", "/")

In [32]:
env = dict(os.environ, PYTHONUNBUFFERED="1")

for test_num, entry in tests.items():

    known_doc = entry['known']
    unknown_doc = entry['unknown']
    print(f"Working on {test_num}: {known_doc} vs {unknown_doc}")
    
    cmd = [
        sys.executable.replace("c:\\", "C:/").replace("\\", "/"), "-u", script_loc.replace("C:\\", "C:/").replace("\\", "/"),
        "--known_loc", known_loc,
        "--unknown_loc", unknown_loc,
        "--metadata_loc", metadata_loc,
        "--model_loc", model_loc,
        "--save_loc", save_loc,
        "--corpus", corpus,
        "--data_type", data_type,
        "--known_doc", known_doc,
        "--unknown_doc", unknown_doc,
        "--openai_model", openai_model,
        "--max_tokens", str(max_tokens),
        "--temperature", str(temperature),
        "--n", str(n_samples),
    ]

    subprocess.run(cmd, text=True, env=env, check=True)

Working on test_01: benjamin_rogers_mail_4 vs benjamin_rogers_mail_1
Working on problem: benjamin_rogers_mail_4 vs benjamin_rogers_mail_1
Loading model
Loading data
Getting common n-grams
Generating paraphrases


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Scoring phrases
    Scoring known text
Processing Phrase - phrase_01
Processing Phrase - phrase_02
Processing Phrase - phrase_03
Processing Phrase - phrase_04
Processing Phrase - phrase_05
Processing Phrase - phrase_06
Processing Phrase - phrase_07
Processing Phrase - phrase_08
Processing Phrase - phrase_09
Processing Phrase - phrase_10
Processing Phrase - phrase_11
Processing Phrase - phrase_12
Processing Phrase - phrase_13
Processing Phrase - phrase_14
Processing Phrase - phrase_15
Processing Phrase - phrase_16
Processing Phrase - phrase_17
Processing Phrase - phrase_18
Processing Phrase - phrase_19
Processing Phrase - phrase_20
Processing Phrase - phrase_21
Processing Phrase - phrase_22
Processing Phrase - phrase_23
Processing Phrase - phrase_24
Processing Phrase - phrase_25
Processing Phrase - phrase_26
Processing Phrase - phrase_27
Processing Phrase - phrase_28
Processing Phrase - phrase_29
Processing Phrase - phrase_30
Processing Phrase - phrase_31
Processing Phrase - phrase_32
P

/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning:

Working on test_02: jeffrey_shankman_mail_4 vs joannie_williamson_mail_4
Working on problem: jeffrey_shankman_mail_4 vs joannie_williamson_mail_4
Loading model
Loading data
Getting common n-grams
Generating paraphrases


...

/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning:

Working on test_10: darron_giron_mail_3 vs david_delainey_mail_3
Working on problem: darron_giron_mail_3 vs david_delainey_mail_3
Loading model
Loading data
Getting common n-grams
Generating paraphrases


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Scoring phrases
    Scoring known text
Processing Phrase - phrase_01
Processing Phrase - phrase_02
Processing Phrase - phrase_03
Processing Phrase - phrase_04
Processing Phrase - phrase_05
Processing Phrase - phrase_06
Processing Phrase - phrase_07
Processing Phrase - phrase_08
Processing Phrase - phrase_09
Processing Phrase - phrase_10
Processing Phrase - phrase_11
Processing Phrase - phrase_12
Processing Phrase - phrase_13
Processing Phrase - phrase_14
Processing Phrase - phrase_15
Processing Phrase - phrase_16
Processing Phrase - phrase_17
Processing Phrase - phrase_18
Processing Phrase - phrase_19
Processing Phrase - phrase_20
Processing Phrase - phrase_21
Processing Phrase - phrase_22
Processing Phrase - phrase_23
Processing Phrase - phrase_24
Processing Phrase - phrase_25
Processing Phrase - phrase_26
Processing Phrase - phrase_27
Processing Phrase - phrase_28
Processing Phrase - phrase_29
Processing Phrase - phrase_30
Processing Phrase - phrase_31
Processing Phrase - phrase_32
P

/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[obj_cols] = df[obj_cols].applymap(_clean_cell)
/Users/user/Documents/GitHub/av_distributions/scripts/run_openai_paraphrase_method.py:40: FutureWarning: